In [7]:
!pip install langchain-groq
!pip install faiss-cpu
!pip install langchain-community

import os

os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

# Initialize Groq LLM
llm = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    temperature=0.7
)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [9]:
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
) #instanciamos el modelo

prompt = ChatPromptTemplate.from_template("tell me a curious fact about {soccer_player}") #utilizamos el prompt template

output_parser = StrOutputParser() #instanciamos el parses

chain = prompt | llm | output_parser #Realizamos la chain

chain.invoke({"soccer_player": "Ronaldo"}) #hacemos el invoke dandole los valores que necesita el prompt templete

'One curious fact about Cristiano Ronaldo is that he has a museum dedicated to himself in his hometown of Funchal, Madeira, Portugal. The museum, called the "CR7 Museum," features a vast collection of his trophies, medals, and memorabilia, including his Ballon d\'Or awards, UEFA Champions League medals, and even his first football boots. The museum was opened in 2013 and has become a popular tourist destination, with thousands of visitors each year. It\'s a testament to Ronaldo\'s incredible career and his dedication to preserving his legacy for his fans.'

In [10]:
for s in chain.stream({"soccer_player": "Ronaldo"}): #esta es la manera de que la respuesta vaya apareciendo de a poco, es una menera de simular una conversacion humana
    if hasattr(s, 'content'):
        print(s.content, end="", flush=True)
    else:
        print(s, end="", flush=True)

Here's a curious fact about Cristiano Ronaldo: 

Cristiano Ronaldo is known for his intense training regimen and dedication to fitness. However, what's less well-known is that he has a cryotherapy chamber in his home. Cryotherapy involves exposure to extremely low temperatures (often below -100°C) to aid in muscle recovery and reduce inflammation. Ronaldo uses this chamber to help his body recover from intense matches and training sessions, which is just one example of the extreme measures he takes to maintain his physical fitness and stay at the top of his game.

LCEL Chains/Runnables are used with:
 * chain.**invoke**(): call the chain on an input.

    Recibimos la repuesta del llm en el instante
 * chain.**stream**(): call the chain on an input and stream back chunks of the response.

    Recibimos la respuesta del llm con una cierta pausa de manera tal que paresca una converzacion con un humano

 * chain.**batch**(): call the chain on a list of inputs.

    Podemos realizar la llamada en parallelo

Runnables
son elementos que pueden ser utilizados en una chain

RunnableParallel: correo los procesos en parallelo

RunnableLambda: hay que darlñe una entrada de parametros que se los pasar a una funcion lamda y el output sera la salida de la funcion

RunnablePassthrough: toma la entrada y la devuelve de manera identica, lo que entra sale

In [11]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Definir la plantilla de prompt
prompt = ChatPromptTemplate.from_template("Summarize the sensor data: {sensor_id}")

# Definir el parser de salida
output_parser = StrOutputParser()

# Función para calcular el promedio de las lecturas del sensor
def calculate_average(sensor_data):
    readings = sensor_data["readings"]
    average = sum(readings) / len(readings)
    sensor_data["average"] = average
    print(sensor_data)
    return sensor_data

# Función para encontrar la lectura máxima del sensor
def find_max_reading(sensor_data):
    readings = sensor_data["readings"]
    max_reading = max(readings)
    sensor_data["max_reading"] = max_reading
    print(sensor_data)
    return sensor_data

# Función para encontrar la lectura mínima del sensor
def find_min_reading(sensor_data):
    readings = sensor_data["readings"]
    min_reading = min(readings)
    sensor_data["min_reading"] = min_reading
    print(sensor_data)
    return sensor_data

# Definir la cadena de ejecución con tres runnables en paralelo
chain = RunnableParallel(
    #calculamos el average, la lectura maxima y minima
    {
        "average": RunnableLambda(calculate_average),
        "max_reading": RunnableLambda(find_max_reading),
        "min_reading": RunnableLambda(find_min_reading),
        "sensor_id": RunnablePassthrough(id)
    }
) | prompt | llm | output_parser #El runnableparallel procesa tres runnableLambda y un runnanlePassthrough en paralelo por lo cual tendremos cuatro salidas que seran la entrada al promp

# Invocar la cadena con datos de entrada
result = chain.invoke({
    "id": "Sensor_01",
    "readings": [23.4, 25.1, 22.8, 24.5, 23.9]
})

# Imprimir los resultados
print(result)

{'id': 'Sensor_01', 'readings': [23.4, 25.1, 22.8, 24.5, 23.9], 'average': 23.939999999999998}
{'id': 'Sensor_01', 'readings': [23.4, 25.1, 22.8, 24.5, 23.9], 'average': 23.939999999999998, 'max_reading': 25.1}
{'id': 'Sensor_01', 'readings': [23.4, 25.1, 22.8, 24.5, 23.9], 'average': 23.939999999999998, 'max_reading': 25.1, 'min_reading': 22.8}
Here's a summary of the sensor data:

**Sensor ID:** Sensor_01
**Readings:** 5 data points (23.4, 25.1, 22.8, 24.5, 23.9)
**Average Reading:** 23.94
**Maximum Reading:** 25.1
**Minimum Reading:** 22.8

Let me know if you'd like me to help with anything else.


In [ ]:
from langchain_community.vectorstores import FAISS #base de datos vectorial
from langchain_core.output_parsers import StrOutputParser #el parser de datos
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import OpenAIEmbeddings
#falta la APIKEY de openIA

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 10,000 Alumni from all continents and top companies"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)


retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

retrieval_chain.invoke("who are the Alumni of AI Accelera?")

In [ ]:

from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings #falta la APIkey de openIA

vectorstore = FAISS.from_texts(
    ["AI Accelera has trained more than 5,000 Enterprise Alumni."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = ( #los itemgettet permiten que los runnables tomen un solo item (una sola entrada)
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many Enterprise Alumni has trained AI Accelera?", "language": "Pirate English"})